<a href="https://colab.research.google.com/github/Loryana/OCProjet7/blob/main/Projet7_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re

In [ ]:
train_df = df_scaled[df_scaled['TARGET'].notnull()]
test_df = df_scaled[df_scaled['TARGET'].isnull()]
print("Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

train_x, train_y = train_df[feats], train_df['TARGET']
valid_x, valid_y = train_df[feats], train_df['TARGET']
#clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],eval_metric= 'auc')


In [ ]:
from sklearn.metrics import confusion_matrix
# Score métier
def custom_metric(y, y_pred):

    # Matrice de Confusion
    mat_conf = confusion_matrix(y,y_pred)

    TP = mat_conf[0, 0]
    FP = mat_conf[0, 1]
    FN = mat_conf[1, 0]
    TN = mat_conf[1, 1]

    score = (10*FN + FP)

    return score

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer, precision_score,recall_score
from sklearn.metrics import make_scorer
# création des scores avec make_scorer()
custom_scorer = make_scorer(custom_metric, greater_is_better=False)

f1_scorer = make_scorer(fbeta_score, beta=1, greater_is_better=True)
f2_scorer = make_scorer(fbeta_score, beta=2, greater_is_better=True)

precision_scorer = make_scorer(precision_score, greater_is_better=True)
recall_scorer = make_scorer(recall_score, greater_is_better=True)
roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True, needs_proba=True)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
def model_eval_score(model, X_test, y_test, df_scores=None, grid = True):

    y_pred = model.predict(X_test) #returns a single discrete category that it has predicted
    y_pred_proba = model.predict_proba(X_test)[:,1] #returns continuous values that represent the likelihood of each input belonging to each class

    #on calcule la matrice de confusion et on l'affiche
    conf_mat = confusion_matrix(y_test, y_pred)
    print(conf_mat)

    if grid == False:
        dict_score_func = {'Modèle' : type(model).__name__,
                       'Accuracy score': (accuracy_score(y_test, y_pred)),
                       'Recall score': (recall_score(y_test, y_pred)),
                       'Precision score': (precision_score(y_test, y_pred)),
                       'F1 score': (f1_score(y_test, y_pred)),
                       'F2 score': (fbeta_score(y_test, y_pred, beta= 2)),
                       'Score metier':  (custom_metric(y_test, y_pred)),
                       'ROCAUC score': (roc_auc_score(y_test, y_pred_proba)),
                       'FN': conf_mat[1, 0],
                       'FP': conf_mat[0, 1],
                      }

        list_columns = ['Modèle','Accuracy score','Recall score','Precision score',
                    'F1 score','F2 score','Score metier','ROCAUC score',
                   'FN', 'FP']

        df_scores_rs = pd.DataFrame()

    if grid == True:

        df_scores_rs = pd.DataFrame(model.cv_results_).sort_values(by ='rank_test_score', ascending = True).reset_index(drop=True)
        df_scores_rs = df_scores_rs[['rank_test_score','mean_fit_time','param_classifier','params', 'split0_test_score',
                             'split1_test_score', 'split2_test_score',
                             'mean_test_score', 'std_test_score']]

        dict_score_func = {'Modèle' : model.best_estimator_,
                           'Accuracy score': (accuracy_score(y_test, y_pred)),
                           'Recall score': (recall_score(y_test, y_pred)),
                           'Precision score': (precision_score(y_test, y_pred)),
                            'F1 score': (f1_score(y_test, y_pred)),
                           'F2 score': (fbeta_score(y_test, y_pred, beta= 2)),
                           'Score metier':  (custom_metric(y_test, y_pred)),
                           'ROCAUC score': (roc_auc_score(y_test, y_pred_proba)),
                           'FN': conf_mat[1, 0],
                           'FP': conf_mat[0, 1],
                           'fit time': df_scores_rs.loc[0,'mean_fit_time']
                      }

        list_columns = ['Modèle','Accuracy score','Recall score','Precision score',
                    'F1 score','F2 score','Score metier','ROCAUC score',
                   'FN', 'FP','fit time']


    if df_scores is None:
        df_scores = pd.DataFrame(columns = list_columns)

    df_scores = pd.concat([df_scores, pd.DataFrame(dict_score_func, index = [0])]).reset_index(drop=True)

    return df_scores , df_scores_rs

In [ ]:
from sklearn.ensemble import RandomForestClassifier
mod1 = RandomForestClassifier(random_state=27)
mod1.fit(train_x, train_y)
df_scores, df0 = model_eval_score(mod1, valid_x, valid_y, grid = False)

df_scores

In [ ]:
def model_eval(model, X_test, y_test):

    y_pred = model.predict(X_test) #returns a single discrete category that it has predicted
    y_pred_proba = model.predict_proba(X_test)[:,1] #returns continuous values that represent the likelihood of each input belonging to each class

    print(f'Accuracy score    : {accuracy_score(y_test, y_pred):.3}')
    print(f'Recall score      : {recall_score(y_test, y_pred):.3}')
    print(f'Precision score   : {precision_score(y_test, y_pred):.3}')
    print(f'F1 score          : {f1_score(y_test, y_pred):.3}')
    print(f'F2 score          : {fbeta_score(y_test, y_pred, beta=2):.3}')
    print(f'Score metier      : {custom_metric(y_test, y_pred)}')
    print(f'ROCAUC score      : {roc_auc_score(y_test, y_pred_proba):.3}')
    print()

    plt.figure(figsize=(6,4))

    visualizer = ROCAUC(model, binary=True, random_state=101)

    visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
    visualizer.score(X_test, y_test)        # Evaluate the model on the test data

    visualizer.show()